In [ ]:
# NEED TO RUN disentangled_rnn_train_clean
# Do this for different values of "condition_type" in disentangled_rnn_params & over several seeds

In [ ]:
import os
import pickle
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize
import itertools
from itertools import repeat 
from disentangled_rnn_utils import DotDict as Dd

your_path = 'YOUR PATH'

import seaborn
seaborn.set_style(style='white')

In [ ]:
import re
def find_most_recent(file_list, must_contain=None, cant_contain=None, recent=-1):
    """
    Accepts a list of strings of format X_n[.Y optional], returns highest number n
    Each of the strings needs to contain one of must_contain and can't contain any of cant_contain
    """
    # Find all iteration numbers from file list where files match and sort them
    iter_numbers = [int(re.findall(r'\d+', x)[0]) for x in file_list
                    if (True if cant_contain is None else not any([y in x for y in cant_contain]))
                    and (True if must_contain is None else any([y in x for y in must_contain]))]
    iter_numbers.sort()
    # Index is the latest iteration, or None if no iterations were found at all
    index = None if len(iter_numbers) == 0 else str(np.unique(iter_numbers)[recent])
    return index

In [ ]:
date = '2024-08-23'  # YOUR DATE FOR SUMMARIES
x_intercepts_2 = []
oversamples = []
oversamples_2 = []
mirs_corner_both = []
mirs_corr = []
mirs_corr_2 = []
for run in os.listdir(your_path + date):
    path = your_path + date + '/' + run + '/save'
    if len(os.listdir(path)) > 1:
        param = Dd(np.load(path + '/params.npy', allow_pickle=True).item())
        index = find_most_recent(os.listdir(path), ['.pickle'], ['par'])  
        with open(path + '/mirs_all_' + index + '.pickle', 'rb') as handle:
            mir = pickle.load(handle)

        # find run of particular point
        if param.data.condition_type == 'corner_cut_both' and mir[2][7][-1] < 0.5 and mir[2][-1] < 0.01:
            continue

        if mir[2][-1] > 0.08:
            continue
        if param.data.condition_type == 'corner_cut_both':
            x_intercepts_2.append(param.data.intercept_x)
            mirs_corner_both.append(mir)
        elif param.data.condition_type == 'oversample_diagonal_2':
            oversamples.append(param.data.prop_batch_oversample)
            mirs_corr.append(mir)
        elif param.data.condition_type == 'oversample_diagonals':
            oversamples_2.append(param.data.prop_batch_oversample)
            mirs_corr_2.append(mir)
        else:
            raise ValueError(param.data.condition_type)
            

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

mir_index = 0
source_mi_index = -1

corner_both_source_multi_info = np.array([x[2][source_mi_index] for x in mirs_corner_both])
corner_both_ncmi = np.array([x[2][mir_index][-1] for x in mirs_corner_both])

corr_source_multi_info = np.array([x[2][source_mi_index] for x in mirs_corr])
corr_ncmi = np.array([x[2][mir_index][-1] for x in mirs_corr])

corr_2_source_multi_info = np.array([x[2][source_mi_index] for x in mirs_corr_2])
corr_2_ncmi = np.array([x[2][mir_index][-1] for x in mirs_corr_2])

noise_stds = [0.08, 0.04, 0.04, 0.08]
for (X_train, y_train, color, label, noise_std) in zip([np.abs(corner_both_source_multi_info[:, None]), 
                                                        np.abs(corr_source_multi_info[:, None]),
                                                        np.abs(corr_2_source_multi_info[:, None])], 
                                                       [corner_both_ncmi,
                                                         corr_ncmi,
                                                        corr_2_ncmi], 
                                                       ['orange', 'green', 'purple'],
                                                       ['Corner Cut Both', 'Diagonal', 'Both diagonals'],
                                                       noise_stds):
    try:
        start = X_train.min()
        stop = X_train.max()
        kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-3, 1e2))
        gaussian_process = GaussianProcessRegressor(kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9)
        
        gaussian_process.fit(X_train, y_train)
        gaussian_process.kernel_
        
        X = np.linspace(start=start, stop=stop, num=1_000).reshape(-1, 1)
        mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)
        
        plt.scatter(X_train, y_train, label=label, c=color, s=10)
        plt.plot(X, mean_prediction, c=color) # label="Mean prediction",
        plt.fill_between( X.ravel(), mean_prediction - 1.96 * std_prediction, mean_prediction + 1.96 * std_prediction, alpha=0.2, color=color) #label=r"95% confidence interval"
    except ValueError:
        pass
plt.ylim(0,1)
plt.legend(fontsize=16)
plt.xlabel("Normalised source multiinformation", fontsize=16)
plt.ylabel("Linear Conditional InfoM", fontsize=16)

plt.savefig('modularirt_vs_corr')